In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("goszakup_tenders_full_async.csv")

In [2]:
df

,ID,Наименование объявления,Ссылка,Лотов,Организатор,Способ,Начало приема заявок,Окончание приема заявок,"Сумма, тг.",Статус,...,Общие_Вид предмета закупок,Общие_Организатор,Общие_Юр. адрес организатора,Общие_Признаки,Организатор_ФИО представителя,Организатор_Должность,Организатор_E-Mail,Общие_Способ несостоявшейся закупки,Общие_Приглашенный поставщик,Организатор_Создатель объявления
0,15741303-1,услуги по обслуживанию и метрологии технически...,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Республиканское государственное учреждение ""Уп...",Государственные закупки с применением особого ...,2025-11-27 09:00:00,2025-11-27 18:00:00,0.00,Опубликовано,...,Услуга,010440016110 Республиканское государственное у...,"196843100, 040719, Казахстан, г. Караой, ул. В...",['Без учета НДС'],Сыдыков Алишер Ибраимжанович,Сотрудник,NaN,NaN,NaN,NaN
1,15741300-1,вывоз твердо-бытовых отходов,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Республиканское государственное учреждение ""Уп...",Государственные закупки с применением особого ...,2025-11-27 09:00:00,2025-11-27 18:00:00,0.00,Опубликовано,...,Услуга,010440016110 Республиканское государственное у...,"196843100, 040719, Казахстан, г. Караой, ул. В...",['Без учета НДС'],Сыдыков Алишер Ибраимжанович,Сотрудник,NaN,NaN,NaN,NaN
2,15741294-1,приобретение запасных частей к автотранспорту,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Республиканское государственное учреждение ""Уп...",Государственные закупки с применением особого ...,2025-11-27 09:00:00,2025-11-27 18:00:00,0.00,Опубликовано,...,Товар,010440016110 Республиканское государственное у...,"196843100, 040719, Казахстан, г. Караой, ул. В...",['Без учета НДС'],Сыдыков Алишер Ибраимжанович,Сотрудник,NaN,NaN,NaN,NaN
3,15741277-1,приобретение соломы для служебных собак,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Республиканское государственное учреждение ""Уп...",Государственные закупки с применением особого ...,2025-11-27 09:00:00,2025-11-27 18:00:00,0.00,Опубликовано,...,Товар,010440016110 Республиканское государственное у...,"196843100, 040719, Казахстан, г. Караой, ул. В...",['Без учета НДС'],Сыдыков Алишер Ибраимжанович,Сотрудник,NaN,NaN,NaN,NaN
4,15741262-1,приобретение продуктов питания (молоко коровье...,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Республиканское государственное учреждение ""Уп...",Государственные закупки с применением особого ...,2025-11-27 09:00:00,2025-11-27 18:00:00,0.00,Опубликовано,...,Товар,010440016110 Республиканское государственное у...,"196843100, 040719, Казахстан, г. Караой, ул. В...",['Без учета НДС'],Сыдыков Алишер Ибраимжанович,Сотрудник,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15727503-1,Подписка на печатные периодические издания,https://goszakup.gov.kz/ru/announce/index/1572...,1,"Коммунальное государственное учреждение ""Район...",Запрос ценовых предложений,2025-11-20 09:15:13,2025-11-24 09:20:26,1 071 428.57,Опубликовано (прием ценовых предложений),...,Товар,990440005653 Коммунальное государственное учре...,"Казахстан, 196830100, Алматинская область, Или...",['Без учета НДС'],БУДУГУЛОВА КАРЛЫГАШ ТУРАНОВНА,Директор,ilibibl2004@mail.ru,NaN,NaN,NaN
9996,15727502-1,Приобретение прочих запасов,https://goszakup.gov.kz/ru/announce/index/1572...,1,Коммунальное государственное учреждение «Детск...,Запрос ценовых предложений,2025-11-20 09:00:00,2025-11-24 09:05:55,2 500.00,Опубликовано (прием ценовых предложений),...,Товар,170740024769 Коммунальное государственное учре...,"554630100, 140500, Казахстан, г. г.Иртышск, ул...","['Без учета НДС', 'Закупка среди организаций и...",КАЖАХАТОВ БАУРЖАН ГАБДУЛЛИНОВИЧ,Директор,kgkpsportirt@mail.ru,NaN,NaN,NaN
9997,15727501-1,"приобретение картриджа принтера ""Кэнон""",https://goszakup.gov.kz/ru/announce/index/1572...,1,"Государственное учреждение ""Отдел образования ...",Запрос ценовых предложений,2025-11-20 08:55:00,2025-11-24 08:55:00,39 552.00,Опубликовано (прием цено

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   ID                                   10000 non-null  object
 1   Наименование объявления              10000 non-null  object
 2   Ссылка                               10000 non-null  object
 3   Лотов                                10000 non-null  int64 
 4   Организатор                          10000 non-null  object
 5   Способ                               10000 non-null  object
 6   Начало приема заявок                 10000 non-null  object
 7   Окончание приема заявок              10000 non-null  object
 8   Сумма, тг.                           10000 non-null  object
 9   Статус                               10000 non-null  object
 10  Детали_Номер объявления              10000 non-null  object
 11  Детали_Наименование объявления       10000

In [32]:
# ЯЧЕЙКА 1: Базовые импорты

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

In [33]:
# ЯЧЕЙКА 3: Очистка и приведение колонки "Сумма, тг." к числу

price_col = "Сумма, тг."

def clean_price(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    # убираем пробелы
    s = s.replace(" ", "")
    # убираем все, кроме цифр, точки и запятой
    s = ''.join(ch for ch in s if ch.isdigit() or ch in [".", ","])
    # заменяем запятую на точку
    s = s.replace(",", ".")
    if s == "" or s == ".":
        return np.nan
    try:
        return float(s)
    except ValueError:
        return np.nan

df[price_col] = df[price_col].apply(clean_price)

df[price_col].describe()

,"Сумма, тг."
count,1.000000e+04
mean,1.001011e+07
std,3.045023e+08
min,0.000000e+00
25%,3.800000e+04
50%,1.162723e+05
75%,4.307836e+05
max,2.894170e+10


In [34]:
# ЯЧЕЙКА 4: Удаляем строки, где цена NaN или <= 0

initial_rows = len(df)
df = df[df[price_col].notna()]
df = df[df[price_col] > 0]

print(f"Было строк: {initial_rows}")
print(f"Осталось строк после фильтра по цене: {len(df)}")

Было строк: 10000
Осталось строк после фильтра по цене: 9948


In [35]:
# ЯЧЕЙКА 5: Парсим даты (по необходимости)

date_cols = [
    "Начало приема заявок",
    "Окончание приема заявок",
    "Детали_Дата публикации",
    "Детали_Срок начала приема",
    "Детали_Срок окончания приема"
]

for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)

df[date_cols].head()

/tmp/ipython-input-1594854847.py:13: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)
/tmp/ipython-input-1594854847.py:13: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)
/tmp/ipython-input-1594854847.py:13: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)


,Начало приема заявок,Окончание приема заявок,Детали_Дата публикации,Детали_Срок начала приема,Детали_Срок окончания приема
5,2025-11-24 09:00:00,2025-11-26 09:00:00,2025-11-23 17:37:08,2025-11-24 09:00:00,2025-11-26 09:00:00
6,2025-11-24 09:00:00,2025-11-26 09:00:00,2025-11-23 15:49:40,2025-11-24 09:00:00,2025-11-26 09:00:00
13,2025-11-24 09:00:00,2025-11-26 09:00:00,2025-11-22 22:45:51,2025-11-24 09:00:00,2025-11-26 09:00:00
14,2025-12-01 09:10:22,2025-12-16 09:10:32,2025-11-22 18:10:52,2025-12-01 09:10:22,2025-12-16 09:10:32
15,2025-12-01 09:01:00,2025-12-17 09:02:00,2025-11-22 18:29:42,2025-12-01 09:01:00,2025-12-17 09:02:00


In [36]:
# ЯЧЕЙКА 6: Подготовка текста

text_col = "Наименование объявления"

# на всякий случай заменим NaN на пустую строку
df[text_col] = df[text_col].fillna("")

# можно добавить также Детали_Наименование, если хочешь объединить
if "Детали_Наименование объявления" in df.columns:
    df["full_title"] = (
        df[text_col].astype(str) + " " +
        df["Детали_Наименование объявления"].astype(str)
    )
    text_for_tfidf = df["full_title"]
else:
    text_for_tfidf = df[text_col]

In [37]:
# ЯЧЕЙКА 7: TF-IDF векторизация

tfidf = TfidfVectorizer(
    # простой вариант, без кастомных токенайзеров для русского
    max_df=0.8,        # игнорируем слишком частые слова
    min_df=5,          # игнорируем очень редкие слова
    ngram_range=(1, 2) # учитываем униграммы и биграммы
)

X_tfidf = tfidf.fit_transform(text_for_tfidf)

X_tfidf.shape

(9948, 3043)

In [38]:
# ЯЧЕЙКА 8: Функция поиска топ-N похожих тендеров по тексту

def get_similar_tenders(idx, top_n=10):
    """
    idx - индекс строки в df
    top_n - сколько похожих записей вернуть
    """
    # берем вектор конкретного тендера
    vec = X_tfidf[idx]
    # считаем косинусную близость со всеми
    sims = cosine_similarity(vec, X_tfidf).flatten()
    # сортируем по убыванию
    similar_idx = sims.argsort()[::-1]
    # убираем сам объект (idx)
    similar_idx = [i for i in similar_idx if i != idx]
    # выбираем top_n
    top_idx = similar_idx[:top_n]

    result = df.iloc[top_idx].copy()
    result["similarity"] = sims[top_idx]
    return result

# Пример: берем тендер с индексом 0 и смотрим 5 самых похожих
similar_to_0 = get_similar_tenders(0, top_n=5)
similar_to_0[[text_col, price_col, "similarity"]]

,Наименование объявления,"Сумма, тг.",similarity
3952,Услуги по финансовому лизингу,2.590960e+07,1.0
2889,Услуги по финансовому лизингу,6.650266e+08,1.0
2812,Услуги по финансовому лизингу,1.970883e+09,1.0
510,Услуги по финансовому лизингу,9.866779e+07,1.0
261,Услуги по финансовому лизингу,1.917714e+09,1.0


In [43]:
# ЯЧЕЙКА 9: Обучение KMeans на TF-IDF

n_clusters = 30  # можно поиграться этим параметром

kmeans = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10
)
df["cluster"] = kmeans.fit_predict(X_tfidf)

df[["cluster", text_col]].head()

,cluster,Наименование объявления
5,21,Услуги по финансовому лизингу
6,4,автострахование( Внимательно прочитайте технич...
13,12,Приобретение хозяйственных товаров
14,6,"Приобретение продуктов питания - мясо говяжье,..."
15,4,Приобретение товаров из средств родительской о...


In [44]:
# ЯЧЕЙКА 10: Распределение тендеров по кластерам

cluster_counts = df["cluster"].value_counts().sort_index()
cluster_counts

,count
cluster,
0,39
1,498
2,87
3,156
4,5241
5,4
6,91
7,180
8,52


In [45]:
# ЯЧЕЙКА 11: Расчёт IQR по кластерам и флаг аномальной цены

def calc_iqr_flags(df, price_col="Сумма, тг.", cluster_col="cluster"):
    grouped = df.groupby(cluster_col)[price_col]

    stats = grouped.quantile([0.25, 0.5, 0.75]).unstack()
    stats.columns = ["q1", "median", "q3"]
    stats["iqr"] = stats["q3"] - stats["q1"]
    stats["upper_bound"] = stats["q3"] + 1.5 * stats["iqr"]

    # join обратно к df
    df = df.join(stats, on=cluster_col)

    df["price_outlier_high"] = df[price_col] > df["upper_bound"]

    return df, stats

df, cluster_price_stats = calc_iqr_flags(df, price_col=price_col, cluster_col="cluster")

df[["cluster", price_col, "q1", "median", "q3", "upper_bound", "price_outlier_high"]].head()

,cluster,"Сумма, тг.",q1,median,q3,upper_bound,price_outlier_high
5,21,3.538397e+08,53385.71,186965.34,797500.00,1913671.435,True
6,4,2.700000e+04,38000.00,120700.00,495000.00,1180500.000,False
13,12,2.670000e+04,17375.00,43345.00,101550.00,227812.500,False
14,6,7.295210e+06,68700.00,270000.00,1284571.05,3108377.625,True
15,4,3.819484e+07,38000.00,120700.00,495000.00,1180500.000,True


In [46]:
# ЯЧЕЙКА 12: Сколько аномально дорогих тендеров

df["price_outlier_high"].value_counts()

,count
price_outlier_high,
False,8463
True,1485


In [47]:
# ЯЧЕЙКА 13: Признаки риска (rule-based)

# 1) факт приглашённого поставщика
invited_col = "Общие_Приглашенный поставщик"
df["has_invited_supplier"] = df[invited_col].notna()

# 2) подозрительный способ закупки (пример - подставь реальные значения)
suspicious_methods = [
    "Закупка из одного источника",
    "Из одного источника",
    "У одного источника"
]

method_col = "Общие_Способ проведения закупки"

df["suspicious_method"] = df[method_col].isin(suspicious_methods)

# 3) очень короткий период приёма заявок (например, < 3 дней)
if "Начало приема заявок" in df.columns and "Окончание приема заявок" in df.columns:
    df["tender_duration_days"] = (
        (df["Окончание приема заявок"] - df["Начало приема заявок"])
        .dt.total_seconds() / (24 * 3600)
    )
    df["short_duration"] = df["tender_duration_days"] < 3
else:
    df["short_duration"] = False

# 4) базовый риск-скоринг (чем больше факторов, тем выше риск)
df["risk_score"] = (
    df["price_outlier_high"].astype(int) * 3 +
    df["has_invited_supplier"].astype(int) * 2 +
    df["suspicious_method"].astype(int) * 2 +
    df["short_duration"].astype(int) * 1
)

df[["ID", text_col, price_col, "cluster", "price_outlier_high",
    "has_invited_supplier", "suspicious_method", "short_duration", "risk_score"]].head()

,ID,Наименование объявления,"Сумма, тг.",cluster,price_outlier_high,has_invited_supplier,suspicious_method,short_duration,risk_score
5,15741196-1,Услуги по финансовому лизингу,3.538397e+08,21,True,True,False,True,6
6,15741170-1,автострахование( Внимательно прочитайте технич...,2.700000e+04,4,False,True,False,True,3
13,15741029-1,Приобретение хозяйственных товаров,2.670000e+04,12,False,True,False,True,3
14,15740956-1,"Приобретение продуктов питания - мясо говяжье,...",7.295210e+06,6,True,False,False,False,3
15,15740951-1,Приобретение товаров из средств родительской о...,3.819484e+07,4,True,False,False,False,3


In [48]:
# ЯЧЕЙКА 14: Отбор подозрительных тендеров

risk_threshold = 4
suspicious_df = df[df["risk_score"] >= risk_threshold].copy()

print(f"Подозрительных тендеров: {len(suspicious_df)}")

suspicious_df.sort_values("risk_score", ascending=False).head(20)[
    ["ID", text_col, price_col, "risk_score",
     "price_outlier_high", "has_invited_supplier", "suspicious_method", "short_duration"]
]

Подозрительных тендеров: 256


,ID,Наименование объявления,"Сумма, тг.",risk_score,price_outlier_high,has_invited_supplier,suspicious_method,short_duration
5,15741196-1,Услуги по финансовому лизингу,3.538397e+08,6,True,True,False,True
261,15740265-1,Услуги по финансовому лизингу,1.917714e+09,6,True,True,False,True
696,15739730-1,формирования расчета норматива расхода,3.000000e+06,6,True,True,False,True
513,15739937-1,Предоставление транспортных средств путем фина...,1.995232e+09,6,True,True,False,True
1232,15738908-1,Приобретение 10 –ти квартир для социально уязв...,4.238000e+07,6,True,True,False,True
2812,15736379-1,Услуги по финансовому лизингу,1.970883e+09,6,True,True,False,True
2960,15736199-1,Услуги по финансовому лизингу на приобретение ...,2.065287e+09,6,True,True,False,True
2246,15737103-1,"Услуги по финансовому лизингу КГУ ""Лесное хозя...",2.150567e+08,6,True,True,False,True
2590,15736673-1,"Услуги по финансовому лизингу КГУ ""Асу-Булакск...",7.345250e+07,6,True,True,False,True
8784,15728961-1,Государственные закупки услуг финансового лиз...,2.894170e+10,6,True,True,False,True


In [49]:
df["lower_bound"] = df["q1"] - 1.5 * df["iqr"]
df["price_outlier_low"] = df[price_col] < df["lower_bound"]

In [51]:
cluster_sizes = df.groupby("cluster").size().rename("cluster_size")
df = df.join(cluster_sizes, on="cluster")

# считаем price_outlier_high только если в кластере достаточно объектов
min_cluster_size = 10
df.loc[df["cluster_size"] < min_cluster_size, "price_outlier_high"] = False

In [52]:
invited_col = "Общие_Приглашенный поставщик"

df[invited_col] = df[invited_col].fillna("").astype(str).str.strip()
bad_values = {"", "0", "-", "нет", "нету", "отсутствует"}

df["has_invited_supplier"] = ~df[invited_col].str.lower().isin(bad_values)

In [53]:
method_col = "Общие_Способ проведения закупки"

df[method_col] = df[method_col].fillna("")

pattern = "одного источника"
df["suspicious_method"] = df[method_col].str.lower().str.contains(pattern)

In [54]:
df["tender_duration_days"] = (
    (df["Окончание приема заявок"] - df["Начало приема заявок"])
    .dt.total_seconds() / (24 * 3600)
)

df["short_duration"] = df["tender_duration_days"] < 3
df.loc[df["tender_duration_days"].isna(), "short_duration"] = False
df.loc[df["tender_duration_days"] < 0, "short_duration"] = True  # явный странный кейс

In [55]:
import joblib
from dataclasses import dataclass, field

@dataclass
class TenderRiskModel:
    price_col: str = "Сумма, тг."
    text_col: str = "Наименование объявления"
    invited_col: str = "Общие_Приглашенный поставщик"
    method_col: str = "Общие_Способ проведения закупки"
    start_date_col: str = "Начало приема заявок"
    end_date_col: str = "Окончание приема заявок"
    n_clusters: int = 30
    risk_threshold: int = 4

    tfidf: TfidfVectorizer = field(default=None, init=False)
    kmeans: KMeans = field(default=None, init=False)
    cluster_price_stats_: pd.DataFrame = field(default=None, init=False)
    suspicious_methods: list = field(default_factory=lambda: ["одного источника"], init=False)

    def _prepare_price(self, df):
        # предполагаем, что ты уже почистил цену в исходном df
        return df

    def fit(self, df: pd.DataFrame):
        df = df.copy()
        df = self._prepare_price(df)

        # 1) TF-IDF по тексту
        text = df[self.text_col].fillna("")
        self.tfidf = TfidfVectorizer(max_df=0.8, min_df=5, ngram_range=(1, 2))
        X_tfidf = self.tfidf.fit_transform(text)

        # 2) Кластеризация
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42, n_init=10)
        df["cluster"] = self.kmeans.fit_predict(X_tfidf)

        # 3) IQR по кластерам
        grouped = df.groupby("cluster")[self.price_col]
        stats = grouped.quantile([0.25, 0.5, 0.75]).unstack()
        stats.columns = ["q1", "median", "q3"]
        stats["iqr"] = stats["q3"] - stats["q1"]
        stats["upper_bound"] = stats["q3"] + 1.5 * stats["iqr"]
        stats["lower_bound"] = stats["q1"] - 1.5 * stats["iqr"]
        self.cluster_price_stats_ = stats

        return self

    def _add_features(self, df: pd.DataFrame):
        df = df.copy()

        # текст -> кластеры
        text = df[self.text_col].fillna("")
        X_tfidf = self.tfidf.transform(text)
        df["cluster"] = self.kmeans.predict(X_tfidf)

        # join IQR
        stats = self.cluster_price_stats_
        df = df.join(stats, on="cluster")

        # price outliers
        df["price_outlier_high"] = df[self.price_col] > df["upper_bound"]
        df["price_outlier_low"] = df[self.price_col] < df["lower_bound"]

        # invited supplier
        df[self.invited_col] = df[self.invited_col].fillna("").astype(str).str.strip()
        bad_values = {"", "0", "-", "нет", "нету", "отсутствует"}
        df["has_invited_supplier"] = ~df[self.invited_col].str.lower().isin(bad_values)

        # suspicious method
        df[self.method_col] = df[self.method_col].fillna("").astype(str)
        pattern = "одного источника"
        df["suspicious_method"] = df[self.method_col].str.lower().str.contains(pattern)

        # duration
        if self.start_date_col in df.columns and self.end_date_col in df.columns:
            start = pd.to_datetime(df[self.start_date_col], errors="coerce", dayfirst=True)
            end = pd.to_datetime(df[self.end_date_col], errors="coerce", dayfirst=True)
            df["tender_duration_days"] = (end - start).dt.total_seconds() / (24*3600)
            df["short_duration"] = df["tender_duration_days"] < 3
            df.loc[df["tender_duration_days"].isna(), "short_duration"] = False
            df.loc[df["tender_duration_days"] < 0, "short_duration"] = True
        else:
            df["short_duration"] = False

        return df

    def predict(self, df: pd.DataFrame):
        """
        Возвращает df с колонками:
        price_outlier_high, has_invited_supplier, suspicious_method,
        short_duration, risk_score, risk_flag
        """
        df_feat = self._add_features(df)

        df_feat["risk_score"] = (
            df_feat["price_outlier_high"].astype(int) * 3 +
            df_feat["has_invited_supplier"].astype(int) * 2 +
            df_feat["suspicious_method"].astype(int) * 2 +
            df_feat["short_duration"].astype(int) * 1
        )

        df_feat["risk_flag"] = df_feat["risk_score"] >= self.risk_threshold

        return df_feat

In [56]:
# обучаем на всём датасете (после очистки цены)
risk_model = TenderRiskModel(
    price_col="Сумма, тг.",
    text_col="Наименование объявления",
    n_clusters=30,
    risk_threshold=4
)

risk_model.fit(df)

import joblib
joblib.dump(risk_model, "tender_risk_model.pkl")

['tender_risk_model.pkl']

In [57]:
df

,ID,Наименование объявления,Ссылка,Лотов,Организатор,Способ,Начало приема заявок,Окончание приема заявок,"Сумма, тг.",Статус,...,iqr,upper_bound,price_outlier_high,has_invited_supplier,suspicious_method,tender_duration_days,short_duration,lower_bound,price_outlier_low,cluster_size
5,15741196-1,Услуги по финансовому лизингу,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Коммунальное государственное учреждение ""Марка...",Из одного источника по несостоявшимся закупкам,2025-11-24 09:00:00,2025-11-26 09:00:00,3.538397e+08,Опубликовано,...,744114.29,1.913671e+06,True,True,True,2.000000,True,-1.062786e+06,False,855
6,15741170-1,автострахование( Внимательно прочитайте технич...,https://goszakup.gov.kz/ru/announce/index/1574...,1,Коммунальное государственное учреждение «Средн...,Из одного источника по несостоявшимся закупкам,2025-11-24 09:00:00,2025-11-26 09:00:00,2.700000e+04,Опубликовано,...,457000.00,1.180500e+06,False,True,True,2.000000,True,-6.475000e+05,False,5241
13,15741029-1,Приобретение хозяйственных товаров,https://goszakup.gov.kz/ru/announce/index/1574...,1,"Коммунальное государственное учреждение ""Олент...",Из одного источника по несостоявшимся закупкам,2025-11-24 09:00:00,2025-11-26 09:00:00,2.670000e+04,Опубликовано,...,84175.00,2.278125e+05,False,True,True,2.000000,True,-1.088875e+05,False,224
14,15740956-1,"Приобретение продуктов питания - мясо говяжье,...",https://goszakup.gov.kz/ru/announce/index/1574...,3,Коммунальное государственное казенное предприя...,"Конкурс по приобретению товаров, связанных с о...",2025-12-01 09:10:22,2025-12-16 09:10:32,7.295210e+06,Опубликовано,...,1215871.05,3.108378e+06,True,False,False,15.000116,False,-1.755107e+06,False,91
15,15740951-1,Приобретение товаров из средств родительской о...,https://goszakup.gov.kz/ru/announce/index/1574...,36,Государственное коммунальное казенное предприя...,"Конкурс по приобретению товаров, связанных с о...",2025-12-01 09:01:00,2025-12-17 09:02:00,3.819484e+07,Опубликовано,...,457000.00,1.180500e+06,True,False,False,16.000694,False,-6.475000e+05,False,5241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15727503-1,Подписка на печатные периодические издания,https://goszakup.gov.kz/ru/announce/index/1572...,1,"Коммунальное государственное учреждение ""Район...",Запрос ценовых предложений,2025-11-20 09:15:13,2025-11-24 09:20:26,1.071429e+06,Опубликовано (прием ценовых предложений),...,138968.27,3.921024e+05,True,False,False,4.003623,False,-1.637707e+05,False,474
9996,15727502-1,Приобретение прочих запасов,https://goszakup.gov.kz/ru/announce/index/1572...,1,Коммунальное государственное учреждение «Детск...,Запрос ценовых предложений,2025-11-20 09:00:00,2025-11-24 09:05:55,2.500000e+03,Опубликовано (прием ценовых предложений),...,146362.50,3.869062e+05,False,False,False,4.004109,False,-1.985438e+05,False,108
9997,15727501-1,"приобретение картриджа принтера ""Кэнон""",https://goszakup.gov.kz/ru/announce/index/1572...,1,"Государственное учреждение ""Отдел образования ...",Запрос ценовых предложений,2025-11-20 08:55:00,2025-11-24 08:55:00,3.955200e+04,Опубликовано (прием ценовых предложений),...,217470.00,5.596750e+05,False,False,False,4.000000,False,-3.102050e+05,False,39
9998,15727500-1,Повторная закупка. Услуги по техническому обсл...,https://goszakup.gov.kz/ru/announce/index/1572...,1,"Товарищество с ограниченной ответственностью ""...",Запрос ценовых предложений,2025-11-21 10:00:00,2025-11-25 10:00:00,2.100000e+04,Опубликовано (прием ценовых предложений),...,744114.29,1.913671e+06,False,False,False,4.000000,False,-1.062786e+06,False,855


In [59]:
# ЯЧЕЙКА: добавляем risk_score, risk_flag, risk_level и сразу смотрим аутпут

def add_risk_outputs(df, risk_threshold=4):
    """
    df – DataFrame с колонками:
      price_outlier_high, has_invited_supplier, suspicious_method, short_duration
    risk_threshold – порог, начиная с которого тендер считается подозрительным
    """
    df = df.copy()

    # Если risk_score уже есть, можно пересчитать или оставить как есть.
    df["risk_score"] = (
        df["price_outlier_high"].astype(int) * 3 +
        df["has_invited_supplier"].astype(int) * 2 +
        df["suspicious_method"].astype(int) * 2 +
        df["short_duration"].astype(int) * 1
    )

    # Бинарный флаг "подозрительный тендер"
    df["risk_flag"] = df["risk_score"] >= risk_threshold

    # Текстовый уровень риска
    def score_to_level(s):
        if s >= 6:
            return "высокий"
        elif s >= 4:
            return "средний"
        elif s >= 1:
            return "низкий"
        else:
            return "нет"

    df["risk_level"] = df["risk_score"].apply(score_to_level)

    return df

# применяем функцию
df_with_risk = add_risk_outputs(df, risk_threshold=4)

# смотрим первые строки с основными аутпутами модели
cols_to_show = [
    "ID",
    "Наименование объявления",
    "Сумма, тг.",
    "cluster",
    "risk_score",
    "risk_level",
    "risk_flag",
    "price_outlier_high",
    "has_invited_supplier",
    "suspicious_method",
    "short_duration",
]

df_with_risk[cols_to_show].head(20)

,ID,Наименование объявления,"Сумма, тг.",cluster,risk_score,risk_level,risk_flag,price_outlier_high,has_invited_supplier,suspicious_method,short_duration
5,15741196-1,Услуги по финансовому лизингу,3.538397e+08,21,8,высокий,True,True,True,True,True
6,15741170-1,автострахование( Внимательно прочитайте технич...,2.700000e+04,4,5,средний,True,False,True,True,True
13,15741029-1,Приобретение хозяйственных товаров,2.670000e+04,12,5,средний,True,False,True,True,True
14,15740956-1,"Приобретение продуктов питания - мясо говяжье,...",7.295210e+06,6,3,низкий,False,True,False,False,False
15,15740951-1,Приобретение товаров из средств родительской о...,3.819484e+07,4,3,низкий,False,True,False,False,False
16,15740949-1,Приобретение продуктов питания - масло сливочн...,4.037058e+06,6,3,низкий,False,True,False,False,False
17,15740946-1,Приобретение продуктов питания - молочная прод...,6.224210e+06,6,3,низкий,False,True,False,False,False
18,15740941-1,"Приобретение продуктов питания - конфеты, печенье",1.334000e+06,6,0,нет,False,False,False,False,False
19,15740917-1,Бензин АИ-92 по талонной системе (бессрочные)....,4.214200e+05,23,5,средний,True,False,True,True,True
20,15740877-1,Приобретение продуктов питания - бакалея,3.734369e+06,6,3,низкий,False,True,False,False,False


In [60]:
df_with_risk.to_csv("df_with_risk.csv", index=False)

In [61]:
import json
import pandas as pd
import numpy as np

# Пример входного JSON (как будто его прислал curl на бэкенд)
request_json = {
    "tenders": [
        {
            "id": "15741196-1",
            "name": "Услуги по финансовому лизингу",
            "price": 353839700.0,
            "organizer": "ГУ «Госорган какой-то»",
            "invited_supplier": "ТОО «Рога и Копыта»",
            "method": "Закупка из одного источника",
            "start_date": "2024-02-10",
            "end_date": "2024-02-11"
        },
        {
            "id": "TEST-2",
            "name": "Приобретение хозяйственных товаров",
            "price": 270000.0,
            "organizer": "ГУ «Другое ведомство»",
            "invited_supplier": "",
            "method": "Открытый конкурс",
            "start_date": "2024-03-01",
            "end_date": "2024-03-15"
        }
    ]
}

print("Входной JSON (как будто с curl):")
print(json.dumps(request_json, ensure_ascii=False, indent=2))

Входной JSON (как будто с curl):
{
  "tenders": [
    {
      "id": "15741196-1",
      "name": "Услуги по финансовому лизингу",
      "price": 353839700.0,
      "organizer": "ГУ «Госорган какой-то»",
      "invited_supplier": "ТОО «Рога и Копыта»",
      "method": "Закупка из одного источника",
      "start_date": "2024-02-10",
      "end_date": "2024-02-11"
    },
    {
      "id": "TEST-2",
      "name": "Приобретение хозяйственных товаров",
      "price": 270000.0,
      "organizer": "ГУ «Другое ведомство»",
      "invited_supplier": "",
      "method": "Открытый конкурс",
      "start_date": "2024-03-01",
      "end_date": "2024-03-15"
    }
  ]
}


In [64]:
from datetime import datetime

def score_tenders_like_api(request_json,
                           risk_threshold=4,
                           suspicious_pattern="одного источника"):
    """
    request_json: dict вида {"tenders": [ {...}, {...} ]}
    возвращает dict с результатами скоринга
    """

    # 1) Превращаем входной JSON в DataFrame в "формате датасета"
    records = []
    for t in request_json["tenders"]:
        records.append({
            "ID": t["id"],
            "Наименование объявления": t["name"],
            "Сумма, тг.": t["price"],
            "Общие_Приглашенный поставщик": t.get("invited_supplier"),
            "Общие_Способ проведения закупки": t.get("method"),
            "Начало приема заявок": t.get("start_date"),
            "Окончание приема заявок": t.get("end_date"),
        })

    df_req = pd.DataFrame(records)

    # приводим даты
    for col in ["Начало приема заявок", "Окончание приема заявок"]:
        df_req[col] = pd.to_datetime(df_req[col], errors="coerce", dayfirst=True)

    # 2) Кластеризация новых тендеров
    text = df_req["Наименование объявления"].fillna("")
    X_tfidf_req = tfidf.transform(text)
    df_req["cluster"] = kmeans.predict(X_tfidf_req)

    # 3) Подтягиваем IQR-статистику по кластерам
    stats = cluster_price_stats[["q1", "median", "q3", "iqr", "upper_bound"]]
    df_req = df_req.join(stats, on="cluster")

    # 4) Флаги и скоринг
    price_col = "Сумма, тг."

    df_req["price_outlier_high"] = df_req[price_col] > df_req["upper_bound"]

    # invited supplier
    invited_col = "Общие_Приглашенный поставщик"
    df_req[invited_col] = df_req[invited_col].fillna("").astype(str).str.strip()
    bad_values = {"", "0", "-", "нет", "нету", "отсутствует"}
    df_req["has_invited_supplier"] = ~df_req[invited_col].str.lower().isin(bad_values)

    # suspicious method
    method_col = "Общие_Способ проведения закупки"
    df_req[method_col] = df_req[method_col].fillna("").astype(str)
    df_req["suspicious_method"] = df_req[method_col].str.lower().str.contains(
        suspicious_pattern
    )

    # duration
    df_req["tender_duration_days"] = (
        (df_req["Окончание приема заявок"] - df_req["Начало приема заявок"])
        .dt.total_seconds() / (24 * 3600)
    )
    df_req["short_duration"] = df_req["tender_duration_days"] < 3
    df_req.loc[df_req["tender_duration_days"].isna(), "short_duration"] = False
    df_req.loc[df_req["tender_duration_days"] < 0, "short_duration"] = True

    # risk_score
    df_req["risk_score"] = (
        df_req["price_outlier_high"].astype(int) * 3 +
        df_req["has_invited_supplier"].astype(int) * 2 +
        df_req["suspicious_method"].astype(int) * 2 +
        df_req["short_duration"].astype(int) * 1
    )

    df_req["risk_flag"] = df_req["risk_score"] >= risk_threshold

    def score_to_level(s):
        if s >= 6:
            return "high"
        elif s >= 4:
            return "medium"
        elif s >= 1:
            return "low"
        else:
            return "none"

    df_req["risk_level"] = df_req["risk_score"].apply(score_to_level)

    # 5) Собираем JSON-ответ в стиле API
    response = {"tenders": []}

    for _, row in df_req.iterrows():
        response["tenders"].append({
            "id": row["ID"],
            "risk_score": int(row["risk_score"]),
            "risk_flag": bool(row["risk_flag"]),
            "risk_level": row["risk_level"],
            "features": {
                "price": float(row[price_col]),
                "cluster": int(row["cluster"]),
                "price_outlier_high": bool(row["price_outlier_high"]),
                "has_invited_supplier": bool(row["has_invited_supplier"]),
                "suspicious_method": bool(row["suspicious_method"]),
                "short_duration": bool(row["short_duration"]),
                "q1": float(row["q1"]) if not np.isnan(row["q1"]) else None,
                "median": float(row["median"]) if not np.isnan(row["median"]) else None,
                "q3": float(row["q3"]) if not np.isnan(row["q3"]) else None,
                "upper_bound": float(row["upper_bound"]) if not np.isnan(row["upper_bound"]) else None,
            }
        })

    return response

In [65]:
response_json = score_tenders_like_api(request_json, risk_threshold=4)

print("JSON-ответ модели (как будто от REST API):")
print(json.dumps(response_json, ensure_ascii=False, indent=2))

JSON-ответ модели (как будто от REST API):
{
  "tenders": [
    {
      "id": "15741196-1",
      "risk_score": 7,
      "risk_flag": true,
      "risk_level": "high",
      "features": {
        "price": 353839700.0,
        "cluster": 21,
        "price_outlier_high": true,
        "has_invited_supplier": true,
        "suspicious_method": true,
        "short_duration": false,
        "q1": 53385.71,
        "median": 186965.34,
        "q3": 797500.0,
        "upper_bound": 1913671.435
      }
    },
    {
      "id": "TEST-2",
      "risk_score": 3,
      "risk_flag": false,
      "risk_level": "low",
      "features": {
        "price": 270000.0,
        "cluster": 12,
        "price_outlier_high": true,
        "has_invited_supplier": false,
        "suspicious_method": false,
        "short_duration": false,
        "q1": 17375.0,
        "median": 43345.0,
        "q3": 101550.0,
        "upper_bound": 227812.5
      }
    }
  ]
}
